In [1]:
!nvidia-smi

Sat Mar 11 16:20:15 KST 2023

+-----------------------------------------------------------------------------+
| NVIDIA-SMI 455.32.00    Driver Version: 455.32.00    CUDA Version: 11.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla V100-PCIE...  On   | 00000000:14:00.0 Off |                    0 |
| N/A   32C    P0    24W / 250W |      4MiB / 32510MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
|   1  Tesla V100-PCIE...  On   | 00000000:15:00.0 Off |                    0 |
| N/A   32

In [2]:
import os
import glob

import numpy as np
import matplotlib.pyplot as plt
import torch

from monai.apps import download_and_extract
from monai.config import print_config
from monai.utils import set_determinism

print_config()
set_determinism(0)

import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning) 
warnings.filterwarnings("ignore", category=UserWarning) 

MONAI version: 1.1.0
Numpy version: 1.23.5
Pytorch version: 1.7.1+cu110
MONAI flags: HAS_EXT = False, USE_COMPILED = False, USE_META_DICT = False
MONAI rev id: a2ec3752f54bfc3b40e7952234fbeb5452ed63e3
MONAI __file__: /home/aistore3/.conda/envs/snuimageseg/lib/python3.8/site-packages/monai/__init__.py

Optional dependencies:
Pytorch Ignite version: 0.4.8
Nibabel version: 5.0.0
scikit-image version: 0.19.3
Pillow version: 9.4.0
Tensorboard version: 2.11.0
gdown version: 4.6.0
TorchVision version: 0.8.2+cu110
tqdm version: 4.64.1
lmdb version: 1.4.0
psutil version: 5.9.0
pandas version: 1.5.2
einops version: 0.6.0
transformers version: 4.25.1
mlflow version: 2.1.1
pynrrd version: NOT INSTALLED or UNKNOWN VERSION.

For details about installing the optional dependencies, please visit:
    https://docs.monai.io/en/latest/installation.html#installing-the-recommended-dependencies



In [3]:
directory = "temp"
if not os.path.exists(directory):
    os.makedirs(directory)
root_dir = tempfile.mkdtemp() if directory is None else directory
data_dir = "Pancreas-CT"

In [4]:
train_images = sorted(
    glob.glob(os.path.join(data_dir, "data", "*.nii.gz")))
train_labels = sorted(
    glob.glob(os.path.join(data_dir, "TCIA_pancreas_labels-02-05-2017", "*.nii.gz")))
data_dicts = [
    {"image": image_name, "label": label_name}
    for image_name, label_name in zip(train_images, train_labels)
]
print(len(data_dicts))

82


In [5]:
def nii_loader(filename) :
    import nibabel as nib
    import numpy as np
    import matplotlib.pyplot as plt
    nimg = nib.load( filename )
    return nimg.get_fdata() , nimg.affine, nimg.header

def target_layers(dataset):
    for idx in range(len(dataset)):
        image = dataset[idx]['image']
        label = dataset[idx]['label']

        image_data, image_affine, image_header = nii_loader(image)
        label_data, label_affine, label_header = nii_loader(label)

        max_area = 0
        max_layer = 0

        for current_layer in range(0, image_data.shape[2]):
            current_label = label_data[:,:,current_layer]
            label_area = sum(sum(current_label))
            if label_area > max_area:
                max_area = label_area
                max_layer = current_layer
                
        print(f'{os.path.basename(image)} {max_layer}')

In [6]:
target_layers(data_dicts)

PANCREAS_0001.nii.gz 126
PANCREAS_0002.nii.gz 89
PANCREAS_0003.nii.gz 90
PANCREAS_0004.nii.gz 96
PANCREAS_0005.nii.gz 96
PANCREAS_0006.nii.gz 96
PANCREAS_0007.nii.gz 103
PANCREAS_0008.nii.gz 104
PANCREAS_0009.nii.gz 102
PANCREAS_0010.nii.gz 90
PANCREAS_0011.nii.gz 96
PANCREAS_0012.nii.gz 107
PANCREAS_0013.nii.gz 75
PANCREAS_0014.nii.gz 132
PANCREAS_0015.nii.gz 108
PANCREAS_0016.nii.gz 181
PANCREAS_0017.nii.gz 131
PANCREAS_0018.nii.gz 101
PANCREAS_0019.nii.gz 100
PANCREAS_0020.nii.gz 102
PANCREAS_0021.nii.gz 105
PANCREAS_0022.nii.gz 185
PANCREAS_0023.nii.gz 124
PANCREAS_0024.nii.gz 168
PANCREAS_0025.nii.gz 89
PANCREAS_0026.nii.gz 140
PANCREAS_0027.nii.gz 93
PANCREAS_0028.nii.gz 106
PANCREAS_0029.nii.gz 82
PANCREAS_0030.nii.gz 182
PANCREAS_0031.nii.gz 131
PANCREAS_0032.nii.gz 127
PANCREAS_0033.nii.gz 144
PANCREAS_0034.nii.gz 73
PANCREAS_0035.nii.gz 98
PANCREAS_0036.nii.gz 69
PANCREAS_0037.nii.gz 66
PANCREAS_0038.nii.gz 115
PANCREAS_0039.nii.gz 96
PANCREAS_0040.nii.gz 117
PANCREAS_0041.ni